# DOHMH New York City Restaurant Inspection Results

## Data Cleaning

Our goal is to ...

In [ ]:
columns=['CAMIS',
 'DBA',
 'BORO',
 'BUILDING',
 'STREET',
 'ZIPCODE',
 'PHONE',
 'CUISINE DESCRIPTION',
 'INSPECTION DATE',
 'VIOLATION CODE',
 'CRITICAL FLAG',
 'SCORE',
 'GRADE',
 'GRADE DATE',
 'RECORD DATE',
 'INSPECTION TYPE',
 'BBL',
 'NTA']

ds.select(*columns).head()

In [ ]:
df = ds.select(*columns).to_df()
df.shape

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
from openclean.operator.map.violations import fd_violations
groups = fd_violations(df, ['CAMIS', 'INSPECTION DATE'], 'SCORE')